<a href="https://colab.research.google.com/github/hoangnuthuphuong/Machine_Learning/blob/main/Chuong5_HuanLuyenMoHinh_%26_Loi/C5_4_2_HuanLuyenMoHinhHQLogistic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Bài 1**
Thư viện [scikit-learn](https://scikit-learn.org/stable/) của Python là thư viện chuyên dụng dành cho Học máy (Machine learning). Trong bài tập này, bạn học cách sử dụng hàm **train_test_split** để thực hiện chia tập dữ liệu ban đầu thành các tập dữ liệu huấn luyện.
Cho tập dữ liệu ban đầu $D = ((X, y)| X ∈ R^{m×n}, y ∈ R^m )$
Hãy chia D thành tập dữ liệu huấn luyện (training test) và tập dữ liệu kiểm thử (test set) theo tỉ lệ lần lượt là 70% và 30%.
$$D_{train} = [(X_{train}, y_{train})| X_{train} ∈ R^{m×n}, y_{train} ∈ R^m]$$
$$D_{test} = [(X_{test}, y_{test})| X_{test} ∈ R^{m×n}, y_{test} ∈ R^m]$$
Hãy sử dụng các tập dữ liệu:
- ex1data1.txt và ex1data2.txt

Tham khảo cú pháp hàm **train_test_split** tại đây: [link](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

In [1]:
import numpy as np
import os

In [33]:
def readData(folder, filename, delimiter= ","):
    data = np.loadtxt(os.path.join(folder, filename), delimiter=delimiter)
    X = data[:, :-1]
    y = data[:, -1]
    return X, y

def split_train_test(X, y, test_size, random_state = None):
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state = random_state)
    return X_train, X_test, y_train, y_test

def print_train_test(X_train, X_test, y_train, y_test):
    print("Size of training set\nX train:",X_train.shape)
    print("y train", y_train.shape)
    print("\nSize of test set\nX test:",X_test.shape)
    print("y test:" ,y_test.shape)

def main():
    X, y = readData("/content/drive/MyDrive/Classroom/Học máy 1 Phần cơ sở","ex2data1.txt")
    X_train, X_test, y_train, y_test =  split_train_test(X, y, test_size = 0.3, random_state=4)
    print_train_test(X_train, X_test, y_train, y_test)

if __name__ == "__main__":
    main()

Size of training set
X train: (70, 2)
y train (70,)

Size of test set
X test: (30, 2)
y test: (30,)


# **Bài 2**
Sử dụng phương pháp chia train-test ở bài 1 để thực hiện huấn
luyện và kiểm thử mô hình Hồi quy Logistic mà bạn đã lập trình
ở Bài học trước.
Lưu ý:
- Sử dụng tập dữ liệu ex2data1.txt và ex2data2.txt;
- Cần thêm vector cột có giá trị 1 vào tập X_train và X_test;
- Cần chuyển y_train và y_test thành vector cột dạng ma trận $R^{m×1}$
cho phù hợp với mô tả toán học của bài học trước;
- Thực hiện chuẩn hóa dữ liệu với $X_{train}$ và $X_{test}$;
- Đánh giá hiệu năng của mô hình hồi quy tuyến tính bằng các chỉ số:
$$accuracy(y, \hat{y}) = \frac{1}{m}∑(y = \hat{y})$$

In [18]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [19]:
def readData(folder , filename):
    data = np.loadtxt(os.path.join(folder, filename), delimiter = ',')
    X = data[:, :-1]
    y = data[:, -1].reshape(-1, 1)
    one = np.ones((X.shape[0], 1))
    X = np.concatenate((one, X), axis = 1)
    return X, y

In [20]:
def normScaling(X):
  for col in range(1, X.shape[1]):
    temp = X[:, col]
    X[:, col] = (temp - np.min(temp)) / (np.max(temp) - np.min(temp))

In [21]:
def standardScaling(X):
  for col in range(1, X.shape[1]):
    temp = X[:, col]
    X[:, col] = (temp - np.mean(temp)) / (np.std(temp))

In [22]:
def predict(X, w):
    h_w = 1 / (1 + np.exp(- np.dot(X, w)))
    return (h_w >= 0.5).astype('int32')

In [23]:
def costFunction(X, y ,w):
    m = X.shape[0]
    h_w = 1 / (1 + np.exp(- np.dot(X, w)))
    J_w = (-1/m) * (np.dot(y.T, np.log(h_w)) + np.dot((1-y).T, np.log(1-h_w)))
    return J_w[0, 0]

In [24]:
def gradient(X, y, w):
    m = X.shape[0]
    h_w = 1 / (1 + np.exp(- np.dot(X, w)))
    return (1/m) * np.dot(X.T, h_w - y)

In [25]:
def gradientDescent(X, y, w_init, alpha, n = 1500):
    w_old = w_init.reshape(-1, 1)
    cost_values = []
    for i in range(n):
        w_new = w_old - alpha * gradient(X, y, w_old)
        cost_values.append(costFunction(X, y, w_new))
        w_old = w_new
    return w_new, cost_values

In [29]:
def main():
    # Đọc dữ liệu
    X, y = readData("/content/drive/MyDrive/Classroom/Học máy 1 Phần cơ sở","ex2data1.txt")

    # Chia tập dữ liệu thành training set và test set
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,
                                                        random_state = 5)
    
    # Chuẩn hóa dữ liệu
    standardScaling(X_train)
    standardScaling(X_test)

    # Huấn luyện mô hình bằng gradient descent
    w_init = np.zeros((X_train.shape[1], 1))
    w_opt, loss = gradientDescent(X=X_train,y=y_train, 
                                           w_init=w_init, alpha=0.01, n=1500)
    # Dự đoán
    y_pred = predict(X_test, w_opt)
    
    # Đánh giá hiệu năng của mô hình
    print("ĐÁNH GIÁ HIỆU NĂNG MÔ HÌNH\n")
    print("\tAccuracy: ", accuracy_score(y_test, y_pred))

if __name__ == "__main__":
    main()

ĐÁNH GIÁ HIỆU NĂNG MÔ HÌNH

	Accuracy:  0.9333333333333333


# **Bài 3**
Giải lại bài 2 với yêu cầu sau:
- Thay thuật toán tối ưu Gradient Descent bằng 2 – 3 thuật toán tối ưu khác;
- So sánh hiệu năng của mô hình **hồi quy Logistic** được huấn luyện bởi các thuật toán tối ưu khác nhau này.

In [34]:
import os
import numpy as np
from scipy import optimize
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [35]:
def readData(folder , filename):
    data = np.loadtxt(os.path.join(folder, filename), delimiter = ',')
    X = data[:, :-1]
    y = data[:, -1]
    one = np.ones((X.shape[0], 1))
    X = np.concatenate((one, X), axis = 1)
    return X, y

In [36]:
def normScaling(X, y):
    for col in range(1, X.shape[1]):
        temp = X[:, col]
        X[:, col] = (temp - np.min(temp)) / (np.max(temp) - np.min(temp))
    return X,y

In [37]:
def standardScaling(X, y):
    for col in range(1, X.shape[1]):
        temp = X[:, col]
        X[:, col] = (temp - np.mean(temp)) / (np.std(temp))
    return X,y

In [38]:
def predict(X, w):
    h_w = 1 / (1 + np.exp(- np.dot(X, w)))
    return (h_w >= 0.5).astype('int32')

In [39]:
def costFunction(w, X, y):
    m = X.shape[0]
    h_w = 1 / (1 + np.exp(- np.dot(X, w)))
    J_w = (-1/m) * (np.dot(y, np.log(h_w)) + np.dot((1-y), np.log(1-h_w)))
    return J_w

In [40]:
def logisticRegression(X, y, w_init, method, iterations):
    result = optimize.minimize(fun=costFunction, x0=w_init, args=(X,y),
                               method= method,
                               options={"maxiter":iterations})
    return result.x, result.fun

In [41]:
def sosanh(X_train, y_train, X_test, y_test, algorithms):
    w_init = np.zeros((X_train.shape[1], 1))
    result = {}
    for algorithm in algorithms:
        w, loss = logisticRegression(X_train, y_train, w_init, method = algorithm, iterations=1500)
        y_pred = predict(X_test, w)
        result[algorithm] = accuracy_score(y_pred, y_test)
    return result

In [45]:
def main():
    # Đọc dữ liệu
    X, y = readData("/content/drive/MyDrive/Classroom/Học máy 1 Phần cơ sở","ex2data1.txt")

    # Tách dữ liệu thành training set và test set
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,
                                                        random_state = 5)

    # Chuẩn hóa dữ liệu
    X_train, y_train = standardScaling(X_train, y_train)
    X_test, y_test = standardScaling(X_test, y_test)

    # Huấn luyện mô hình và so sánh mô hình
    '''Các thuật toán tối ưu:
    TNC, BFGS, L-BFGS-B, Nelder-Mead, Powell, CG, Newton-CG, COBYLA, SLSQP, ...'''
    print(sosanh(X_train, y_train, X_test, y_test, ['TNC']))
    print(sosanh(X_train, y_train, X_test, y_test, ['BFGS']))
    print(sosanh(X_train, y_train, X_test, y_test, ['L-BFGS-B']))
    print(sosanh(X_train, y_train, X_test, y_test, ['Nelder-Mead']))
    print(sosanh(X_train, y_train, X_test, y_test, ['Powell']))
    print(sosanh(X_train, y_train, X_test, y_test, ['CG']))
    print(sosanh(X_train, y_train, X_test, y_test, ['COBYLA']))
    print(sosanh(X_train, y_train, X_test, y_test, ['SLSQP']))

if __name__ == "__main__":
    main()

{'TNC': 0.9333333333333333}
{'BFGS': 0.9333333333333333}
{'L-BFGS-B': 0.9333333333333333}
{'Nelder-Mead': 0.9333333333333333}
{'Powell': 0.9333333333333333}
{'CG': 0.9333333333333333}
{'COBYLA': 0.9333333333333333}
{'SLSQP': 0.9333333333333333}


# **Bài 4**
Thư viện **scikit-learn** cung cấp mô hình [LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) để phục vụ xây dựng các mô hình hồi quy logistic. Hãy sử dụng mô hình **LogisticRegression** để giải bài tập 2.
Lưu ý:
- Xét trường hợp có thêm vector cột giá trị 1 vào X
- Xét trường hợp không thêm vector cột giá trị 1 vào X
- Đánh giá hiệu năng của mô hình bằng ***Accuracy***

In [48]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

In [54]:
def readData(folder , filename):
    data = np.loadtxt(os.path.join(folder, filename), delimiter = ',')
    X = data[:, :-1]
    y = data[:, -1]
    one = np.ones((X.shape[0], 1))
    X = np.concatenate((one, X), axis = 1)
    return X, y

In [50]:
def normScaling(X):
  for col in range(1, X.shape[1]):
    temp = X[:, col]
    X[:, col] = (temp - np.min(temp)) / (np.max(temp) - np.min(temp))

In [51]:
def standardScaling(X):
  for col in range(1, X.shape[1]):
    temp = X[:, col]
    X[:, col] = (temp - np.mean(temp)) / (np.std(temp))

In [55]:
def main():

    # Đọc dữ liệu
    X, y = readData("/content/drive/MyDrive/Classroom/Học máy 1 Phần cơ sở","ex2data1.txt")

    # Chia tập dữ liệu thành training set và test set
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,
                                                        random_state = 5)

    # Chuẩn hóa dữ liệu
    standardScaling(X_train)
    standardScaling(X_test)

    # Huấn luyện mô hình bằng gradient descent
    classifier = LogisticRegression()
    classifier.fit(X_train, y_train)
    # Dự đoán
    y_pred = classifier.predict(X_test)

    # Đánh giá hiệu năng của mô hình
    print("ĐÁNH GIÁ HIỆU NĂNG MÔ HÌNH\n")
    print("\tAccuracy: ",accuracy_score(y_test, y_pred))

if __name__ == "__main__":
    main()

ĐÁNH GIÁ HIỆU NĂNG MÔ HÌNH

	Accuracy:  0.9333333333333333


# **Bài 5**
Thư viện **scikit-learn** cung cấp rất nhiều phương thức [tiền xử lý dữ liệu (preprocessing data)](https://scikit-learn.org/stable/modules/preprocessing.html), hãy giải lại bài 4 với yêu cầu bổ sung là:
- Thực hiện chuẩn hóa dữ liệu bằng [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler);
- Thực hiện chuẩn hóa dữ liệu bằng [MinMaxScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html).

In [ ]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [ ]:
def readData(folder , filename):
    data = np.loadtxt(os.path.join(folder, filename), delimiter = ',')
    X = data[:, :-1]
    y = data[:, -1]
    return X, y

In [86]:
def featureScaling(X_train, X_test, y_train, y_test):
    sc_X = StandardScaler()
    X_train = sc_X.fit_transform(X_train)
    X_test = sc_X.transform(X_test)
    return X_train, X_test, y_train, y_test

In [88]:
def main():

    # Đọc dữ liệu
    X, y = readData("/content/drive/MyDrive/Classroom/Học máy 1 Phần cơ sở","ex2data1.txt")

    # Chia tập dữ liệu thành training set và test set
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,
                                                        random_state = 5)

    # Chuẩn hóa dữ liệu
    X_train, X_test, y_train, y_test = featureScaling(X_train, X_test, y_train, y_test)

    # Huấn luyện mô hình
    classifier = LogisticRegression()
    classifier.fit(X_train, y_train)
    
    # Dự đoán
    y_pred = classifier.predict(X_test)

    # Đánh giá hiệu năng của mô hình
    print("ĐÁNH GIÁ HIỆU NĂNG MÔ HÌNH\n")
    print("\tAccuracy: ",accuracy_score(y_test, y_pred))

if __name__ == "__main__":
    main()

ĐÁNH GIÁ HIỆU NĂNG MÔ HÌNH

	Accuracy:  0.9333333333333333


# **Bài 6**
Thư viện scikit-learn cung cấp cách thức thực hiện [cross-validation](https://scikit-learn.org/stable/modules/cross_validation.html). Hãy giải lại bài tập 5 với yêu cầu bổ sung là thực hiện 10-fold cross-validation cho quá trình huấn luyện mô hình.
Gợi ý:
- Sử dụng StandardScaler/MinMaxScaler và mô hình LinearRegression;
- Tạo [10-fold cross validation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html);
- Thực hiện huấn luyện mô hình với [cross_validate](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html);
- Hoặc thực hiện huấn luyenj mô hình với [cross_val_score](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html);
- Sử dụng [cross_val_predict](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html#sklearn.model_selection.cross_val_predict) để dự đoán.

In [97]:
import os
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold, cross_val_predict
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model

In [90]:
def readData(folder , filename):
    data = np.loadtxt(os.path.join(folder, filename), delimiter = ',')
    X = data[:, :-1]
    y = data[:, -1]
    return X, y

In [91]:
def featureScalingSplit(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=15)
    sc_X = StandardScaler()
    X_train = sc_X.fit_transform(X_train)
    X_test = sc_X.transform(X_test)
    return X_train, X_test, y_train, y_test

In [92]:
def kFoldCrossValiation(X_train, y_train):
    kf = KFold(n_splits=10)
    step = 0
    valitdate_models = {}
    for train2_index, val_index in kf.split(X = X_train,y = y_train):
        step = step + 1
        classifier = LogisticRegression()
        X_train2, X_val = X_train[train2_index], X_train[val_index]
        y_train2, y_val = y_train[train2_index], y_train[val_index]
        classifier.fit(X_train2, y_train2)
        y_pred = classifier.predict(X_val)
        valitdate_models[accuracy_score(y_val, y_pred)] =  classifier
    max_accuracy = max(list(valitdate_models.keys()))
    return valitdate_models[max_accuracy]

In [99]:
def validateTestSet(X_test, y_test, classifier):
    print("\nĐánh giá hiệu năng mô hình trên tập dữ liệu test:")
    y_pred = classifier.predict(X_test)
    print("\tAccuracy: ", accuracy_score(y_test, y_pred))
    return accuracy_score(y_test, y_pred)

In [100]:
def main():
    # Đọc dữ liệu
    X, y = readData("/content/drive/MyDrive/Classroom/Học máy 1 Phần cơ sở","ex2data1.txt")

    # Chuẩn hóa dữ liệu
    X_train, X_test, y_train, y_test = featureScalingSplit(X, y)

    # Dự đoán bằng cross_val_predict
    lasso = linear_model.Lasso()
    y_pred = cross_val_predict(lasso, X, y, cv=3)
    print("y predict:\n",y_pred) 
    
    # Huấn luyện mô hình bằng k-Fold Cross Valiation
    classifier = kFoldCrossValiation(X_train, y_train)

    # Đánh giá hiệu năng
    validateTestSet(X_test, y_test, classifier)
    
if __name__ == "__main__":
    main()

y predict:
 [ 0.3416013  -0.06917999  0.30275196  0.73941074  0.85369697  0.241114
  0.85768087  0.50292753  0.94468779  0.58568502  0.66902666  0.33546736
  0.90546071  0.97079845  0.38053261  0.69425935  0.49538968  0.41813546
  0.93631081  0.53741773  0.3708595   0.88304767  0.22916652 -0.01813075
  0.77336768  0.59325874  0.54758935  0.6416448   0.38125848  0.25555563
  0.61234422  0.73931688  0.39801659  0.46339809  0.46146346  0.3945393
  0.70298666  0.80177799  0.47480312  0.44692487  0.72505158  0.30575566
  0.92932078  0.53833597  0.2895612   0.46693274  0.81444878  1.19342091
  0.95485655  1.15438155  0.88186574  0.92643797  0.64851308  0.28779454
  0.3257164   0.42839608  0.99662239  0.65411863  0.77877131  0.84928887
  0.96440002  0.14881348  0.26855987  0.12594545  0.45308002  0.38921229
  0.75224268  0.16974459  1.04603932  0.51286024 -0.14934358  0.6667575
  0.98385923  0.5626135   0.57150257  1.09157877  0.55487202  0.45659869
  0.22004636  0.51942023  0.91395809  0.724